In [1]:
! kaggle

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 6, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [0]:
! cp kaggle.json /root/.kaggle/kaggle.json

In [0]:
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle datasets download ellenyusa/4classimages

 47% 9.00M/19.2M [00:00<00:00, 59.0MB/s]
100% 19.2M/19.2M [00:00<00:00, 94.0MB/s]


In [5]:
! unzip 4classimages.zip

Archive:  4classimages.zip
   creating: data/
   creating: data/cats/
  inflating: data/cats/cat.1.jpg     
  inflating: data/cats/cat.10.jpg    
  inflating: data/cats/cat.100.jpg   
  inflating: data/cats/cat.101.jpg   
  inflating: data/cats/cat.102.jpg   
  inflating: data/cats/cat.103.jpg   
  inflating: data/cats/cat.104.jpg   
  inflating: data/cats/cat.105.jpg   
  inflating: data/cats/cat.106.jpg   
  inflating: data/cats/cat.107.jpg   
  inflating: data/cats/cat.108.jpg   
  inflating: data/cats/cat.109.jpg   
  inflating: data/cats/cat.11.jpg    
  inflating: data/cats/cat.110.jpg   
  inflating: data/cats/cat.111.jpg   
  inflating: data/cats/cat.112.jpg   
  inflating: data/cats/cat.113.jpg   
  inflating: data/cats/cat.114.jpg   
  inflating: data/cats/cat.115.jpg   
  inflating: data/cats/cat.116.jpg   
  inflating: data/cats/cat.117.jpg   
  inflating: data/cats/cat.118.jpg   
  inflating: data/cats/cat.119.jpg   
  inflating: data/cats/cat.12.jpg    
  inflating: data/

In [6]:
import pandas as pd
from keras.models import Model
from keras.layers import Dense, Input, Conv2D, MaxPool2D, Flatten, Dropout, GlobalMaxPool2D
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
generator = ImageDataGenerator(horizontal_flip=True,shear_range=.2,rotation_range=10)

In [8]:
gen = generator.flow_from_directory("data/",target_size=(256,256))

Found 808 images belonging to 4 classes.


In [9]:
gen.class_indices

{'Humans': 0, 'cats': 1, 'dogs': 2, 'horses': 3}

In [10]:
resnet_model = ResNet50(include_top=False,weights="imagenet",input_shape=(256,256,3),classes=len(gen.class_indices))

W0711 11:43:07.490488 140566695896960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0711 11:43:07.546982 140566695896960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0711 11:43:07.558974 140566695896960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0711 11:43:07.610007 140566695896960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0711 11:43:07.611278

94658560/94653016 [==============================] - 3s 0us/step


In [11]:
resnet_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [0]:
#include_top doesnot give the lasts dense layer

In [0]:
for layer in resnet_model.layers:
  layer.trainable = False

In [0]:
flat = Flatten()(resnet_model.output)
d1 = Dense(200,activation="tanh")(flat)
d2 = Dense(100,activation="tanh")(d1)
d3 = Dense(50,activation="tanh")(d2)
out = Dense(len(gen.class_indices),activation="softmax")(d3)

In [16]:
model = Model(inputs = [resnet_model.input],output=[out])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  """Entry point for launching an IPython kernel.


In [17]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

W0711 11:43:38.415156 140566695896960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [19]:
model.fit_generator(gen,epochs=5,steps_per_epoch=10)

W0711 11:43:49.805844 140566695896960 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
10/10 [==============================] - 10s 1s/step - loss: 0.7068 - acc: 0.7625
Epoch 2/5
10/10 [==============================] - 4s 389ms/step - loss: 0.2499 - acc: 0.9187
Epoch 3/5
10/10 [==============================] - 6s 608ms/step - loss: 0.2010 - acc: 0.9348
Epoch 4/5
10/10 [==============================] - 6s 617ms/step - loss: 0.1432 - acc: 0.9563
Epoch 5/5
10/10 [==============================] - 6s 626ms/step - loss: 0.1170 - acc: 0.9688


In [20]:
model.evaluate_generator(gen,steps=10)

[0.13236454129219055, 0.9527027027027027]